In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
import time

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
import seaborn as sns

from pandarallel import pandarallel
pandarallel.initialize()

import lightgbm as lgb
from sklearn.metrics import roc_auc_score,confusion_matrix
from sklearn.model_selection import train_test_split

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
train_app = pd.read_pickle('after_EDA/train_app.pkl')
train_sms = pd.read_pickle('after_EDA/train_sms.pkl')
train_user = pd.read_pickle('after_EDA/train_user.pkl')
train_voc = pd.read_pickle('after_EDA/train_voc.pkl')

test_app = pd.read_pickle('after_EDA/test_app.pkl')
test_sms = pd.read_pickle('after_EDA/test_sms.pkl')
test_user = pd.read_pickle('after_EDA/test_user.pkl')
test_voc = pd.read_pickle('after_EDA/test_voc.pkl')



In [7]:
'''
    对抗性检验，检测是否同一分布
    
'''
gbm = lgb.LGBMClassifier(objective='binary',is_unbalance=True)

def AD_test(flag):
#     import pdb; pdb.set_trace()

    print(flag+":")
    if flag == 'app':
        a = train_app.drop(columns=['month_id','phone_no_m'])
        b = test_app.drop(columns=['month_id','phone_no_m'])
    
    elif flag == 'sms':
        a = train_sms.drop(columns=['request_datetime','phone_no_m','opposite_no_m'])
        b = test_sms.drop(columns=['request_datetime','phone_no_m','opposite_no_m'])
        
    elif flag == 'user':
        a = train_user.copy()
        b = test_user.copy()
        a.drop(columns=['label','arpu_201908','arpu_201909','arpu_201910','arpu_201911','arpu_201912',
                       'arpu_202001','arpu_202002','arpu_202003','phone_no_m','is_arrears','is_null','is_all_null'],inplace=True)
        b.drop(columns=['arpu_202004','phone_no_m','is_arrears','is_null','is_all_null'],inplace=True)
        
        print(a.columns,b.columns)
        
    else:
        a = train_voc.drop(columns=['start_datetime','phone_no_m','opposite_no_m','imei_m'])
        b = test_voc.drop(columns=['start_datetime','phone_no_m','opposite_no_m','imei_m'])
        
    
    
    a['type'] = '0'
    b['type'] = '1'
    c = pd.concat([a,b],axis=0)
    
    
    y = c['type']
    c.drop(columns='type',inplace=True)
    
    for x in c.select_dtypes(include='object').columns:
#         print(x)
        c[x] = c[x].astype('category')
        

    
    
    X_train, X_test, y_train, y_test = train_test_split(c,y,shuffle=True)
    
    gbm.fit(X_train,y_train)
    y_pred = gbm.predict(X_test)
    
    print('ROC Score: ',roc_auc_score(y_test,y_pred))
    
#     tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
#     print('tn: ',tn)
#     print('fp: ',fp)
#     print('fn: ',fn)
#     print('tp: ',tp)
    
#     plt.figure(figsize=(12,6))
#     lgb.plot_importance(gbm, max_num_features=30)
#     plt.title("Featurertances")
#     plt.show()
    


In [8]:
# 除去时间因素， 因为test的都是4月份，而训练集都是前面几个月份
# 出去电话号码，和imei_m编号(能唯一手机的)都去掉，因为数据的时间分布不一样
AD_test('app')
AD_test('user')
AD_test('sms')
AD_test('voc')

app:
ROC Score:  0.5644894609369933
user:
Index(['city_name', 'county_name', 'idcard_cnt', 'mean_arpu'], dtype='object') Index(['city_name', 'county_name', 'idcard_cnt', 'mean_arpu'], dtype='object')
ROC Score:  0.6294317102943171
sms:
ROC Score:  0.5725029948825174
voc:
ROC Score:  0.6004698521832397


In [6]:
train_user

,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label,is_arrears,mean_arpu,is_null,is_all_null
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,46.062500,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,45.000000,0,0,45.132812,0,0
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,79.000000,79.187500,79.125000,79.312500,41.406250,34.093750,59.406250,60.000000,0,0,63.941406,0,0
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,18.843750,24.453125,22.000000,21.406250,22.156250,21.093750,61.312500,63.000000,0,0,31.783203,0,0
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,19.203125,29.656250,130.375000,173.125000,19.000000,19.000000,19.000000,23.203125,0,0,54.070312,0,0
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,0,0,50.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6101,bfc115bb229849f256e52e45cac5c400125cfe24c443f1...,宜宾,高县分公司,1,1.900391,2.500000,1.799805,2.400391,1.799805,1.599609,1.299805,1.400391,0,0,1.837524,0,0
6102,081f09756ed35d349558170213175ceb91b8d1a0392d1d...,天府新区,直管区,4,NaN,NaN,NaN,5.238281,1454.000000,36.062500,22.000000,22.000000,1,0,307.860156,1,0
6103,fbf1d2b9437078cc8b573528034557f7165cc8ce8776de...,未知,未知,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.125000,1,0,83.125000,1,0
6104,aee363f2f1ff659167a2566d7b466fefd27c907a469590...,绵阳,三台分公司,3,269.500000,243.000000,272.500000,246.625000,259.250000,243.250000,243.000000,257.750000,0,0,254.359375,0,0


In [10]:
a = train_app.copy()
a['type'] = 'train'
b = test_app.copy()
b['type'] = 'test'
c = pd.concat([a,b],axis=0)
c.to_csv('app.csv')

In [11]:
a = train_sms.copy()
a['type'] = 'train'
b = test_sms.copy()
b['type'] = 'test'
c = pd.concat([a,b],axis=0)
c.to_csv('sms.csv')

In [12]:
a = train_voc.copy()
a['type'] = 'train'
b = test_voc.copy()
b['type'] = 'test'
c = pd.concat([a,b],axis=0)
c.to_csv('voc.csv')

In [14]:
a = train_user.copy()
a['type'] = 'train'
b = test_user.copy()
b['type'] = 'test'
c = pd.concat([a,b],axis=0)
c.to_csv('user.csv')

In [ ]:
考虑要不要app分类？
考虑voc,sms中，通话时间段，早上晚上？